# 🙀 Cat Evaluation

### 📝 Imports

In [1]:
import torch
from torch import Tensor
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import os

from PIL import Image

from cat_discriminator_neural_net import CatDiscriminatorNeuralNet
from cats_dataset import CatsDataset

### 🔧 Config

In [2]:
captain_image_paths = os.listdir("data/captain-512x512")

for i in range(len(captain_image_paths)):
    captain_image_paths[i] = "data/captain-512x512/" + captain_image_paths[i]

print(captain_image_paths)
image_paths = captain_image_paths
trained_model_path = "trained_networks/cat_discriminator_256x256.pth"

['data/captain-512x512/00aef248cb01448e8b65a7a8c165215c.jpg', 'data/captain-512x512/019a074cb12243188a53572d8f80db7c.jpg', 'data/captain-512x512/029a76d87600461894794a66b2847332.jpg', 'data/captain-512x512/02e8828e28a4473891b25ef41e6c38e5.jpg', 'data/captain-512x512/04fc2a033ec948d48da898aba894389e.jpg', 'data/captain-512x512/0687f594e0964bfc9a7ea65349be0449.jpg', 'data/captain-512x512/09812fd63f8e4f19b0977f0636f0ab0b.jpg', 'data/captain-512x512/09ebfc47a95046cd9491a97c2424505a.jpg', 'data/captain-512x512/0a98d279579347b79e58f7be8e879390.jpg', 'data/captain-512x512/0aab6cd58945459fa23cf3e45bd5ecba.jpg', 'data/captain-512x512/11d9ee7ad03f476d96b0e29b0f888b51.jpg', 'data/captain-512x512/121395684b7344be97307689e472ba6f.jpg', 'data/captain-512x512/12b4cf6cd4de4d1f823244c55f1cc311.jpg', 'data/captain-512x512/177422cfff404233aec28f9866f7ec62.jpg', 'data/captain-512x512/184f6c1c8212428680da0fa33d59b1f7.jpg', 'data/captain-512x512/191df60215f04d72a9e94d056dd27381.jpg', 'data/captain-512x512/1

### 🥾 Initialize the Neural Net

In [3]:
net = CatDiscriminatorNeuralNet()

if os.path.isfile(trained_model_path):
    net.load_state_dict(torch.load(trained_model_path))

net.cuda()

C:\Users\Dan\AppData\Local\Temp\ipykernel_21300\100295788.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(trained_model_path))


RuntimeError: Error(s) in loading state_dict for CatDiscriminatorNeuralNet:
	size mismatch for conv1.weight: copying a param with shape torch.Size([12, 3, 5, 5]) from checkpoint, the shape in current model is torch.Size([12, 3, 10, 10]).
	size mismatch for conv4.weight: copying a param with shape torch.Size([96, 48, 5, 5]) from checkpoint, the shape in current model is torch.Size([96, 48, 3, 3]).
	size mismatch for fc1.weight: copying a param with shape torch.Size([1624, 16224]) from checkpoint, the shape in current model is torch.Size([1624, 80736]).

### 🌐 Create Transforms

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(((0.5), (0.5), 0.5), ((0.5), (0.5), 0.5))
])

def load_image(image_path: str) -> torch.Tensor:
    image = Image.open(image_path)
    image = transform(image)
    image = image.unsqueeze(0) # makes the image a batch, rather than a single image
    return image

images = [load_image(image_path) for image_path in image_paths]

In [ ]:
# Set the model to evaluation mode


net.eval()

def evaluate_image(image_path: str, image: Tensor):
    outputs = net(image.to('cuda'))  # Get raw logits
    probabilities = F.softmax(outputs, dim=1)  # Convert logits to probabilities
    _, predicted = torch.max(outputs, 1)
        
    print(f"Image: {image_path}")
    print(f"Predicted Class: {CatsDataset.classes[predicted.item()]}")
        
    # Iterate over each class to print its probability
    for class_idx, class_name in enumerate(CatsDataset.classes):
        class_prob = probabilities[0][class_idx].item()
        print(f"Probability of '{class_name}': {class_prob:.4f}")
        

with torch.no_grad():
    for index, image in enumerate(images):
        evaluate_image(image_paths[index], image)

Image: data/captain-256x256/04fc2a033ec948d48da898aba894389e.jpg
Predicted Class: captain
Probability of 'bathroom-cat': 0.0000
Probability of 'captain': 1.0000
Probability of 'control': 0.0000
Image: data/captain-256x256/0a98d279579347b79e58f7be8e879390.jpg
Predicted Class: captain
Probability of 'bathroom-cat': 0.0000
Probability of 'captain': 1.0000
Probability of 'control': 0.0000
Image: data/captain-256x256/0aab6cd58945459fa23cf3e45bd5ecba.jpg
Predicted Class: captain
Probability of 'bathroom-cat': 0.0000
Probability of 'captain': 1.0000
Probability of 'control': 0.0000
Image: data/captain-256x256/11d9ee7ad03f476d96b0e29b0f888b51.jpg
Predicted Class: captain
Probability of 'bathroom-cat': 0.0000
Probability of 'captain': 1.0000
Probability of 'control': 0.0000
Image: data/captain-256x256/121395684b7344be97307689e472ba6f.jpg
Predicted Class: captain
Probability of 'bathroom-cat': 0.0001
Probability of 'captain': 0.9999
Probability of 'control': 0.0000
Image: data/captain-256x256/12